# Assignment 1

The goal of this assignment is to supply you with the **building blocks** of **neural networks** (NNs). In this notebook, we will cover the main aspects of NNs, such as **Backpropagation** and **Optimization Methods**. All mathematical operations should be implemented in **NumPy** only. 

The assignmnent consist of two parts:
* *Part one: Blocks* - the place where the main **building blocks** of the NNs are implemented by you
* *Part two: Experiments* - a playground to train the models

### Note
Some of the concepts below have not (yet) been discussed during the lecture. These will be discussed further during the next lectures. 

## Table of contents


* [0. Before you begin](#0.-Before-you-begin)

Part one: Blocks

* [1. Backpropagation](#1.-Backpropagation)   (mandatory)
* [2. Dense layer](#2.-Dense-layer)   (mandatory)
* [3. ReLU nonlinearity](#3.-ReLU-nonlinearity)   (mandatory)
* [4. Sequential model](#4.-Sequential-model)
* [5. Loss](#5.-Loss)   (mandatory)
* [6. $L_2$ Regularization](#6.-$L_2$-Regularization)   (mandatory)
* [7. Optimizer](#7.-Optimizer)
* [8. Advanced blocks](#8.-Advanced-blocks)
    * [8.1 Dropout](#8.1-Dropout)
    * [8.2 MSE Loss](#8.2-MSE-Loss)
    
Part two: Experiments

* [9. Circles Classification Task](#9.-Circles-Classification-Task)
* [10. Digits Classification Task](#10.-Digits-Classification-Task)

# 0. Before you begin

To check whether the code you've written is correct, we'll use **automark**. For this, we created for each of you an account with the username being your student number. 

In [ ]:
import automark as am

# fill in you student number as your username
username = 'your_student_number'

# to check your progress, you can run this function
am.get_progress(username)

So far all your tests are 'not attempted'. At the end of this notebook you'll need to have completed all except 5 tests. The other five will be for next week. At the end of this notebook the output of `am.get_progress(username)` should match the example below.

```
---------------------------------------------
| Your name / student number                |
| your_email@your_domain.whatever           |
---------------------------------------------
| box_blur                 | not attempted  |
| conv_matrix              | not attempted  |
| dense_forward            | completed      |
| dense_grad_W             | completed      |
| dense_grad_b             | completed      |
| dense_grad_input         | completed      |
| dropout_forward          | completed      |
| dropout_grad_input       | completed      |
| flatten_forward          | not attempted  |
| flatten_grad_input       | not attempted  |
| hinge_forward            | completed      |
| hinge_grad_input         | completed      |
| l2_regularizer           | completed      |
| maxpool_forward          | not attempted  |
| mse_forward              | completed      |
| mse_grad_input           | completed      |
| relu_forward             | completed      |
| relu_grad_input          | completed      |
---------------------------------------------
```

Next, we'll import the numpy library:

In [ ]:
from __future__ import print_function, absolute_import, division #You don't need to know what this is. 
import numpy as np #this imports numpy, which is used for vector- and matrix calculations

This paragraph is optional and **solely informational**. 

This notebook makes use of **classes** and their **instances** that we have already implemented for you. It allows us to write less code and make it more readable. If you are interested in it, here are some useful links:
* The official [documentation](https://docs.python.org/3/tutorial/classes.html) 
* Video by *sentdex*: [Object Oriented Programming Introduction](https://www.youtube.com/watch?v=ekA6hvk-8H8)
* Antipatterns in OOP: [Stop Writing Classes](https://www.youtube.com/watch?v=o9pEzgHorH0)

The interface of the current blocks is mostly inspired by **[Torch](http://torch.ch) / [PyTorch](http://pytorch.org)**. You can also take a look at the first implementation of [Keras](https://github.com/fchollet/keras/tree/37a1db225420851cc668600c49697d9a2057f098)

# Part one: Blocks

# 1. Backpropagation

Neural networks consist of several layers. Each layer is a function of several parameters that we call weights: $h = f(x, w)$ where $h$ is the layer, $x$ is a vector of inputs and w is a vector of weights. 
In the neural network, the output of one layer is the input for the next layer. This means we can chain the different functions. The whole neural network $F$ then becomes a composition of different functions. 
$$
F = f_k \circ f_{k-1} \circ \dots \ f_1\\
h_1 = f_1(x, w_1)\\
h_2 = f_2(h_1, w_2)\\
\dots \\
\dot{y} = f_k(h_{k-1}, w_k)
$$
In the above functions, $w_1$ and $w_2$ are different **weight vectors** that apply to the different layers $h_1$ and $h_2$. The weights of a neural network basically determine the effect certain outputs have on the next layer. (Please note: When searching for these terms on the internet, be aware that **weights** are sometimes called **parameters**, and $w$ is sometimes denoted as $\theta$.) 


At the end of every neural network, there is a loss function. A loss function calculates for the performance of the Neural Network. The calculation of this score depends on the task at hand. For classification tasks the loss function would calculate the difference between prediction and the correct value. In this case the function is a summation of this difference for each data point. Calculating this difference can, again, be done in different ways. One example that we have discussed in class is the squared-loss for linear regression. (Here, the difference between predicted and correct classification is squared so positive and negative differences don't cancel eachother.) 
$$\mathcal{L} = \tfrac{1}{2}\sum_{n = 1}^N (y_n - \dot{y}_n)^2$$
Here, $n$ denotes the different datapoints, $y_n$ and $\dot{y}$ represent the correct and the predicted value for that data point respectively. 



The smaller the outcome of this loss function, the better the Neural Network predicts the data. Therefore, we concentrate on **minimizing the loss function** as a means for **training** the neural network. 


Training is done with [Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent). Another word for **gradient** is **derivative**. We use derivatives to update the weights of the neural network to make better predictions. 
The weights of the $k$-th layer are updated according to the following scheme:
$$
w_k \leftarrow w_k - \gamma \frac{\partial \mathcal{L}}{\partial w_k} 
$$
* $\partial f(x)/\partial x$ means the partial derivative of $f(x)$ with respect to $x$. On the internet, derivatives are sometimes written as $f'(x)$, $df/dx$, $\Delta f/x$ or $\nabla f/x$. (Some of these notations have slightly different meanings, but you can ignore that for now.) 
* Hyperparameter $\gamma$ is called the *learning rate* (You'll learn more about hyperparameters later. For now, the only thing you'll have to know is that the value of a hyperparameter is set by you.) 
* Note that $k$ denotes a layer and $n$ denotes a data point.


The computation of $\partial \mathcal{L}/\partial w_k$ is done using the [chain rule](https://en.wikipedia.org/wiki/Chain_rule):
$$
\frac{\partial \mathcal{L}}{\partial w_k} = 
\frac{\partial \mathcal{L}}{\partial h_k}
\frac{\partial h_k}{\partial w_k} = 
\frac{\partial \mathcal{L}}{\partial h_{k+1}}
\frac{\partial h_{k+1}}{\partial h_k}
\frac{\partial h_k}{\partial w_k} = \dots
$$


Therefore, for each layer, we can calculate the following expressions: 
* $h_k = f_k(h_{k-1}, w_k)$ - the forward pass
* $\partial h_{k}/\partial h_{k-1}$ - the partial derivative of the output with respect to the input
* $\partial h_{k}/\partial w_k$ - the partial derivative of the output with respect to the parameters


This whole process of updating weights by calculating the gradient is called [Backpropagation](https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf). Click [here](https://www.youtube.com/watch?v=Ilg3gGewQ5U) for a pretty good video explaining backpropagation. 

# 2. Dense layer
A dense Layer is the basic layer of a neural network. (Other terms for dense layer are fully-connected layer and multiplicative layer.) A dense layer transforms an input matrix of size `(n_objects, d_in)` to a matrix of size `(n_objects, d_out)` (where d stands for dimensions) by performing the following operation:
$$
H = XW + b
$$
Here $H$ represents the function of the dense layer, $X$ is the input matrix, $W$ is the weight matrix for this layer and $b$ is the bias. The bias $b$ is a vector. 

A more detailed version of this function is: 
$$
H_{k,n} = \sum\limits_{i=1}^{d_{in}} X_{n,i}W_{i,k} + b_k
$$
where $n$ denotes again a single data object and $k$ the $k^{th}$ layer.

**Example**: 

You have a neural network of just 1 layer. The inputs are points in a 3D space and you want to classify this point as either $-1$ or $1$. 
You have $75$ objects in your training set. 

Therefore, $X$ has shape $75 \times 3$. $H$ has shape $75 \times 1$. Weight $W$ of the layer has shape $3 \times 1$. 

Using the input for a layer and producing its output is called a forward pass. Here, you'll implement the forward pass:  
$$
H = XW + b
$$

In [ ]:
def dense_forward(x_input, W, b):
    """Perform the mapping of the input
    # Arguments
        x_input: input of a dense layer - np.array of size `(n_objects, n_in)`
        W: np.array of size `(n_in, n_out)`
        b: np.array of size `(n_out,)`
    # Output
        the output of a dense layer 
        np.array of size `(n_objects, b_out)`
    """
    
    #################
    ### YOUR CODE ###
    #################
    return output

Let's check your first function. We set the matrices $X, W, b$:
$$
X = \begin{bmatrix}
1 & -1 \\
-1 & 0 \\
1 & 1 \\
\end{bmatrix} \quad
W = \begin{bmatrix}
4 \\
2 \\
\end{bmatrix} \quad
b = \begin{bmatrix}
3 \\
\end{bmatrix}
$$

And then compute 
$$
XW = \begin{bmatrix}
1 & -1 \\
-1 & 0 \\
1 & 1 \\
\end{bmatrix}
\begin{bmatrix}
4 \\
2 \\
\end{bmatrix} =
\begin{bmatrix}
2 \\
-4 \\
6 \\
\end{bmatrix} \\
XW + b = 
\begin{bmatrix}
5 \\
-1 \\
9 \\
\end{bmatrix} 
$$

In [ ]:
X_test = np.array([[1, -1],
                   [-1, 0],
                   [1, 1]])

W_test = np.array([[4],
                   [2]])

b_test = np.array([3])

h_test =  dense_forward(X_test, W_test, b_test)
print(h_test)

In [ ]:
am.test_student_function(username, dense_forward, ['x_input', 'W', 'b'])

Now, you'll implement a backward pass. As decribed above, this is calculated with the gradient. To calculate the gradient, we'll use the chain rule: 
$$
\frac{\partial \mathcal{L}}{\partial X} = 
\frac{\partial \mathcal{L}}{\partial H}
\frac{\partial H}{\partial X}
$$

In [ ]:
def dense_grad_input(x_input, grad_output, W, b):
    """Calculate the partial derivative of 
        the loss with respect to the input of the layer
    # Arguments
        x_input: input of a dense layer - np.array of size `(n_objects, n_in)`
        grad_output: partial derivative of the loss functions with 
            respect to the ouput of the dense layer 
            np.array of size `(n_objects, n_out)`
        W: np.array of size `(n_in, n_out)`
        b: np.array of size `(n_out,)`
    # Output
        the partial derivative of the loss with 
        respect to the input of the layer
        np.array of size `(n_objects, n_in)`
    """
    #################
    ### YOUR CODE ###
    #################
    return grad_input

In [ ]:
am.test_student_function(username, dense_grad_input, ['x_input', 'grad_output', 'W', 'b'])

Now, instead of computing the gradient with respect to the input, we'll calculate the gradient with respect to the weights and to the bias: 
$$
\frac{\partial \mathcal{L}}{\partial W} = 
\frac{\partial \mathcal{L}}{\partial H}
\frac{\partial H}{\partial W} \\
\frac{\partial \mathcal{L}}{\partial b} = 
\frac{\partial \mathcal{L}}{\partial H}
\frac{\partial H}{\partial b} \\
$$

In [ ]:
def dense_grad_W(x_input, grad_output, W, b):
    """Calculate the partial derivative of 
        the loss with respect to W parameter of the layer
    # Arguments
        x_input: input of a dense layer - np.array of size `(n_objects, n_in)`
        grad_output: partial derivative of the loss functions with 
            respect to the ouput of the dense layer 
            np.array of size `(n_objects, n_out)`
        W: np.array of size `(n_in, n_out)`
        b: np.array of size `(n_out,)`
    # Output
        the partial derivative of the loss 
        with respect to W parameter of the layer
        np.array of size `(n_in, n_out)`
    """
    #################
    ### YOUR CODE ###
    #################
    return grad_W

In [ ]:
am.test_student_function(username, dense_grad_W, ['x_input', 'grad_output', 'W', 'b'])

In [ ]:
def dense_grad_b(x_input, grad_output, W, b):
    """Calculate the partial derivative of 
        the loss with respect to b parameter of the layer
    # Arguments
        x_input: input of a dense layer - np.array of size `(n_objects, n_in)`
        grad_output: partial derivative of the loss functions with 
            respect to the ouput of the dense layer 
            np.array of size `(n_objects, n_out)`
        W: np.array of size `(n_in, n_out)`
        b: np.array of size `(n_out,)`
    # Output
        the partial derivative of the loss 
        with respect to b parameter of the layer
        np.array of size `(n_out,)`
    """
    #################
    ### YOUR CODE ###
    #################
    return grad_b

In [ ]:
am.test_student_function(username, dense_grad_b, ['x_input', 'grad_output', 'W', 'b'])

In [ ]:
am.get_progress(username)

## Dense Layer Class

Here, we define a basic class for the dense layer. You will use this in the Experiments sections below. You don't need to know how this works; we implement it for you, but it is based on the functions you've written above. 

In [ ]:
class Layer(object):
    
    def __init__(self):
        self.training_phase = True
        self.output = 0.0
        
    def forward(self, x_input):
        self.output = x_input
        return self.output
    
    def backward(self, x_input, grad_output):
        return grad_output
    
    def get_params(self):
        return []
    
    def get_params_gradients(self):
        return []

In [ ]:
class Dense(Layer):
    
    def __init__(self, n_input, n_output):
        super(Dense, self).__init__()
        #Randomly initializing the weights from normal distribution
        self.W = np.random.normal(size=(n_input, n_output))
        self.grad_W = np.zeros_like(self.W)
        #initializing the bias with zero
        self.b = np.zeros(n_output)
        self.grad_b = np.zeros_like(self.b)
      
    def forward(self, x_input):
        self.output = dense_forward(x_input, self.W, self.b)
        return self.output
    
    def backward(self, x_input, grad_output):
        # get gradients of weights
        self.grad_W = dense_grad_W(x_input, grad_output, self.W, self.b)
        self.grad_b = dense_grad_b(x_input, grad_output, self.W, self.b)
        # propagate the gradient backwards
        return dense_grad_input(x_input, grad_output, self.W, self.b)
    
    def get_params(self):
        return [self.W, self.b]

    def get_params_gradients(self):
        return [self.grad_W, self.grad_b]

In [ ]:
dense_layer = Dense(2, 1)
x_input = np.random.random((3, 2))
y_output = dense_layer.forward(x_input)
print(x_input)
print(y_output)

# 3. ReLU nonlinearity

The dense layer, from previous section, is linear. Combinging several linear (dense) layers is always equivalent to a single dense layer. Here is the mathematically proof for this: 
$$
H_1 = XW_1 + b_1\\
H_2 = H_1W_2 + b_2\\
H_2 = (XW_1 + b_1)W_2 + b_2 = X(W_1W_2) + (b_1W_2 + b_2) = XW^* + b^*
$$


For this reason, we're also going to need non-linear layers. Non-linear layers ($f$ in the following) are mostly element-wise and hold the following:
$$
H_1 = XW_1 + b_1\\
H_2 = f(H_1)W_2 + b_2\\
H_2 = f(XW_1 + b_1)W_2 + b_2 \neq XW^* + b^*
$$

A popular example of a simple non-linear layer is **ReLU** (Rectified Linear Unit). ReLU doesn't have weights that can be optimized like a dense layer.
$$
\text{ReLU}(x) = \max(0, x)
$$

<img src="./src/relu.png" width="500">

**Example**

$$
\text{ReLU} \Big(
\begin{bmatrix}
1 & -0.5 \\
0.3 & 0.1 
\end{bmatrix}
\Big) = 
\begin{bmatrix}
1 & 0 \\
0.3 & 0.1 
\end{bmatrix}
$$

Next, you will implement the forward pass and backward pass (gradient) for ReLU. 

In [ ]:
def relu_forward(x_input):
    """relu nonlinearity
    # Arguments
        x_input: np.array of size `(n_objects, n_in)`
    # Output
        the output of relu layer
        np.array of size `(n_objects, n_in)`
    """
    #################
    ### YOUR CODE ###
    #################
    return output

In [ ]:
#test forward pass for ReLU, see example above
x_input = np.array([[1, -0.5],
                    [0.3, 0.1]])

print(relu_forward(x_input))

In [ ]:
am.test_student_function(username, relu_forward, ['x_input'])

In [ ]:
def relu_grad_input(x_input, grad_output):
    """relu nonlinearity gradient. 
        Calculate the partial derivative of the loss 
        with respect to the input of the layer
    # Arguments
        x_input: np.array of size `(n_objects, n_in)`
            grad_output: np.array of size `(n_objects, n_in)`
    # Output
        the partial derivative of the loss 
        with respect to the input of the layer
        np.array of size `(n_objects, n_in)`
    """
    #################
    ### YOUR CODE ###
    #################
    return grad_input

In [ ]:
am.test_student_function(username, relu_grad_input, ['x_input', 'grad_output'])

In [ ]:
class ReLU(Layer):
        
    def forward(self, x_input):
        self.output = relu_forward(x_input)
        return self.output
    
    def backward(self, x_input, grad_output):
        return relu_grad_input(x_input, grad_output)

# 4. Sequential model
In order to make the work with layers more comfortable, we create `SequentialNN` - a class, which stores all its layers and performs the basic manipulations. Again, this is for the experiments below and you don't need to know how this works. 

In [ ]:
class SequentialNN(object):

    def __init__(self):
        self.layers = []
        self.training_phase = True
        
    def set_training_phase(self, is_training=True):
        self.training_phase = is_training
        for layer in self.layers:
            layer.training_phase = is_training
        
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, x_input):
        self.output = x_input
        for layer in self.layers:
            self.output = layer.forward(self.output)
        return self.output
    
    def backward(self, x_input, grad_output):
        inputs = [x_input] + [l.output for l in self.layers[:-1]]
        for input_, layer_ in zip(inputs[::-1], self.layers[::-1]):
            grad_output = layer_.backward(input_, grad_output)
            
    def get_params(self):
        params = []
        for layer in self.layers:
            params.extend(layer.get_params())
        return params
    
    def get_params_gradients(self):
        grads = []
        for layer in self.layers:
            grads.extend(layer.get_params_gradients())
        return grads

Here is the simple neural network. It takes an input of shape `(Any, 10)` and passes it through `Dense(10, 4)`, `ReLU` and `Dense(4, 1)`. The output is a batch of size `(Any, 1)`. 
```
  INPUT
    |
Dense(10, 4)
    |
   ReLU
    |
Dense(4, 1)
    |
  OUTPUT
```

In [ ]:
nn = SequentialNN()
nn.add(Dense(10, 4))
nn.add(ReLU())
nn.add(Dense(4, 1))

# 5. Loss

Here we will define the loss functions. Each loss should be able to compute its value and compute its gradient with respect to the input. We have implemented these functions (e.g. forward, backward) for you.  

In [ ]:
class Loss(object):
    
    def __init__(self):
        self.output = 0.0
        
    def forward(self, target_pred, target_true):
        return self.output
    
    def backward(self, target_pred, target_true):
        return np.zeros_like(target_pred)

In the previous section, you've seen a loss function for a linear regression. Here, you'll implement another sort of loss function used in classification: the [Hinge](https://en.wikipedia.org/wiki/Hinge_loss) loss function: 
$$ 
\mathcal{L}(Y, \dot{Y}) = \frac{1}{N}\sum\limits_{n=1}^{N}\max(0, 1 - \dot{y}_n \cdot y_n)
$$

* $N$ represents the number of data points
* $\dot{Y}$ and $Y$ are vectors of length $N$. $Y$ represents the true classes for your data points and $\dot{Y}$ represents your prediction of the classes. 
* $\dot{y}_n$ represents the predicted class of the $n$-th object. $\dot{y}_n \in {\rm I\!R}$
* $y_n$ is the real class of this object. $y_n \in \{-1, 1\}$

This loss function is used in the SVM classifier. You will learn more about SVM in the lectures or click [here](https://www.youtube.com/watch?v=Y6RRHw9uN9o) for a helpful video. 

Now, you'll calculate the forward pass for the Hinge loss function. 

In [ ]:
def hinge_forward(target_pred, target_true):
    """Compute the value of Hinge loss 
        for a given prediction and the ground truth
    # Arguments
        target_pred: predictions - np.array of size `(n_objects,)`
        target_true: ground truth - np.array of size `(n_objects,)`
    # Output
        the value of Hinge loss 
        for a given prediction and the ground truth
        scalar
    """
    #################
    ### YOUR CODE ###
    #################
    return output

You can test your forward pass below. Your output should be: `0.875`

In [ ]:
#test forward pass for Hinge
target_pred = np.array([2,-5,3,0.5])
target_true = np.array([-1,-1,1,1])
print(hinge_forward(target_pred, target_true))

In [ ]:
am.test_student_function(username, hinge_forward, ['target_pred', 'target_true'])

In [ ]:
am.get_progress(username)

Now, you should compute the gradient of the Hinge loss function with respect to its input. The output of this backward pass is a vector with the same shape as the input. To calculate the derivative of a vector, calculate the derivatives of its elements. Those derivatives become the elements of the derivative of the vector: 
$$
\frac{\partial \mathcal{L}}{\partial \dot{Y}} = 
\begin{bmatrix}
\frac{\partial \mathcal{L}}{\partial \dot{y}_1} \\ 
\frac{\partial \mathcal{L}}{\partial \dot{y}_2} \\ 
\vdots \\
\frac{\partial \mathcal{L}}{\partial \dot{y}_N} \\ 
\end{bmatrix}
$$

Now, you'll implement the backward pass for the Hinge loss function. 

In [ ]:
def hinge_grad_input(target_pred, target_true):
    """Compute the partial derivative 
        of Hinge loss with respect to its input
    # Arguments
        target_pred: predictions - np.array of size `(n_objects,)`
        target_true: ground truth - np.array of size `(n_objects,)`
    # Output
        the partial derivative 
        of Hinge loss with respect to its input
        np.array of size `(n_objects,)`
    """

    #################
    ### YOUR CODE ###
    #################  
    return grad_input

In [ ]:
am.test_student_function(username, hinge_grad_input, ['target_pred', 'target_true'])

In [ ]:
class Hinge(Loss):
    
    def forward(self, target_pred, target_true):
        self.output = hinge_forward(target_pred, target_true)
        return self.output
    
    def backward(self, target_pred, target_true):
        return hinge_grad_input(target_pred, target_true)

# 6. $L_2$ Regularization

Loss functions update the weights of your model to improve your predictions. We do this by minimizing the loss function. However, up until now this loss function did not take into account the complexity of your model. Here we mean with complexity the number of parameters that your model stores. We do want to take complexity into account because complex models can perform poorly on test data, while performing excellent on train data. 

To penalize the complextity of the model, we introduce a regularizer. You'll learn more about regularizers in the lectures, but the general idea is that we take the values of the weights into account with the loss function. High values for weights are indicators of complexity. 

There are several ways of adding regularization to a model. We will implement [$L_2$ regularization](http://www.deeplearningbook.org/contents/regularization.html) also known as weight decay:

The key idea of $L_2$ regularization is to add an extra term to the loss functions:
$$
\mathcal{L}^* = \mathcal{L} + \frac{\lambda}{2} \|w\|^2_2
$$

The part we added to the loss function is called the regularization function. 
* $\lambda$ is named weight decay. It is a hyperparameter that determines the influence of the regularization to the outcome of the loss function. 
* $\|w\|^2_2$ is the squared [euclidian norm](https://en.wikipedia.org/wiki/Euclidean_distance) where $\|w\|^2_2 = \|w_1\|^2_2 + \|w_2\|^2_2 ... \|w_k\|^2_2$. 
This function in more detail becomes:

$$
\mathcal{L}^* = \mathcal{L} + \frac{\lambda}{2} \sum\limits_{m=1}^k \|w_m\|^2_2
$$

Because we use a different loss function, the updating of the weights is also slightly changed: 

$$
w_m \leftarrow w_m - \gamma \frac{\partial \mathcal{L}^*}{\partial w_m}\\
\frac{\partial \mathcal{L}^*}{\partial w_m} = \frac{\partial \mathcal{L}}{\partial w_m} + \lambda w_m\\
w_m \leftarrow w_m - \gamma \frac{\partial \mathcal{L}}{\partial w_m} - \lambda w_m
$$

Here, you'll implement the computation of $L_2$: 
$$
L_2(\lambda, [w_1, w_2, \dots, w_k]) = \frac{\lambda}{2} \sum\limits_{m=1}^k \|w_m\|^2_2
$$ 

In [ ]:
def l2_regularizer(weight_decay, weights):
    """Compute the L2 regularization term
    # Arguments
        weight_decay: float
        weights: list of arrays of different shapes
    # Output
        sum of the L2 norms of the input weights
        scalar
    """
    
    #################
    ### YOUR CODE ###
    #################
    return output

You can test your forward pass below. Your output should be: `108.25`

In [ ]:
#test the L2 regularizer
weight_decay = 2
weights = np.array([5,3,7,5,0.5])
print(l2_regularizer(weight_decay, weights))

In [ ]:
am.test_student_function(username, l2_regularizer, ['weight_decay', 'weights'])

# 7. Optimizer

We implement the optimizer to perform the updates of the weights. Here we have implemented two classes that you will need in the Experiments section.   

In [ ]:
class Optimizer(object):
    '''
    This is a basic class. 
    All other optimizers will inherit it
    '''
    def __init__(self, model, lr=0.01, weight_decay=0.0):
        self.model = model
        self.lr = lr
        self.weight_decay = weight_decay
        
    def update_params(self):
        pass


class SGD(Optimizer):
    '''
    Stochastic gradient descent optimizer
    https://en.wikipedia.org/wiki/Stochastic_gradient_descent
    '''
        
    def update_params(self):
        weights = self.model.get_params()
        grads = self.model.get_params_gradients()
        for w, dw in zip(weights, grads):
            update = self.lr * dw + self.weight_decay * w
            # it writes the result to the previous variable instead of copying
            np.subtract(w, update, out=w) 

# 8. Advanced blocks

This is an optional section. If you liked the process of understanding neural networks by implementing them from scratch, here are several more tasks for you.

## 8.1 Dropout

[Dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) is a method of regularization. It can be interpreted as augmentation method. The key idea is to randomly drop some values of the input to avoid overfitting. Its behaviour is different during training and testing.

![dropout](./src/dropout.png)

First of all, you should implement the method of calculating the binary mask. The binary mask has the same shape as the input. The mask could have the value 0 for the certain element with the probability `drop_rate` and it could have the value 1 with a probability of `1.0 - drop_rate`. 

**Note**, the $p$ from the paper linked above is `1.0 - drop_rate`


In [ ]:
def dropout_generate_mask(shape, drop_rate):
    """Generate binary mask 
    # Arguments
        shape: shape of the input array 
            tuple 
        drop_rate: probability of the element 
            to be multiplied by 0
            scalar
    # Output
        binary mask 
    """
    #################
    ### YOUR CODE ###
    #################
    return mask

Now, implement the above-described operation of mapping.

In [ ]:
def dropout_forward(x_input, mask, drop_rate, training_phase):
    """Perform the mapping of the input
    # Arguments
        x_input: input of the layer 
            np.array of size `(n_objects, d_in)`
        mask: binary mask
            np.array of size `(n_objects, d_in)`
        drop_rate: probability of the element to be multiplied by 0
            scalar
        training_phase: bool either `True` - training, or `False` - testing
    # Output
        the output of the dropout layer 
        np.array of size `(n_objects, d_in)`
    """
    #################
    ### YOUR CODE ###
    #################    
    return output

In [ ]:
am.test_student_function(username, dropout_forward, ['x_input', 'mask', 'drop_rate', 'training_phase'])

And, as usual, implement the calculation of the partial derivative of the loss function with respect to the input of a layer. 

In [ ]:
def dropout_grad_input(x_input, grad_output, mask):
    """Calculate the partial derivative of 
        the loss with respect to the input of the layer
    # Arguments
        x_input: input of a dense layer - np.array of size `(n_objects, n_in)`
        grad_output: partial derivative of the loss functions with 
            respect to the ouput of the dropout layer 
            np.array of size `(n_objects, n_in)`
        mask: binary mask
            np.array of size `(n_objects, n_in)`
    # Output
        the partial derivative of the loss with 
        respect to the input of the layer
        np.array of size `(n_objects, n_in)`
    """
    #################
    ### YOUR CODE ###
    #################
    return grad_input

In [ ]:
am.test_student_function(username, dropout_grad_input, ['x_input', 'grad_output', 'mask'])

In [ ]:
class Dropout(Layer):
    
    def __init__(self, drop_rate):
        super(Dropout, self).__init__()
        self.drop_rate = drop_rate
        self.mask = 1.0
        
    def forward(self, x_input):
        if self.training_phase:
            self.mask = dropout_generate_mask(x_input.shape, self.drop_rate)
        self.output = dropout_forward(x_input, self.mask, 
                                      self.drop_rate, self.training_phase)
        return self.output
    
    def backward(self, x_input, grad_output):
        grad_input = dropout_grad_input(x_input, grad_output, self.mask)
        return grad_input

## 8.2 MSE Loss
MSE (Mean Squared Error) is a popular loss for the regression tasks.

$$
\mathcal{L}(Y, \dot{Y}) = \frac{1}{2N}\sum\limits_{n=1}^N(y_n - \dot{y}_n)^2
$$

* $N$ - number of objects
* $\dot{Y}$ and $Y$ are the vectors of length $N$. 
* $\dot{y}_n$ is the predicted target value of the $n$-th object. $\dot{y}_n \in {\rm I\!R}$
* $y_n$ is the real target value of the $n$-th object. $y_n \in {\rm I\!R}$
* This loss function is used to train regressors.

Let's implement the calculation of the loss.

In [ ]:
def mse_forward(target_pred, target_true):
    """Compute the value of MSE loss
        for a given prediction and the ground truth
    # Arguments
        target_pred: predictions - np.array of size `(n_objects,)`
        target_true: ground truth - np.array of size `(n_objects,)`
    # Output
        the value of MSE loss 
        for a given prediction and the ground truth
        scalar
    """
    #################
    ### YOUR CODE ###
    #################  
    return output

In [ ]:
am.test_student_function(username, mse_forward, ['target_pred', 'target_true'])

Now you should compute the gradient of the loss function with respect to its input. 

In [ ]:
def mse_grad_input(target_pred, target_true):
    """Compute the partial derivative 
        of MSE loss with respect to its input
    # Arguments
        target_pred: predictions - np.array of size `(n_objects,)`
        target_true: ground truth - np.array of size `(n_objects,)`
    # Output
        the partial derivative 
        of MSE loss with respect to its input
        np.array of size `(n_objects,)`
    """
    #################
    ### YOUR CODE ###
    #################     
    return grad_input

In [ ]:
am.test_student_function(username, mse_grad_input, ['target_pred', 'target_true'])

In [ ]:
class MSE(Loss):
    
    def forward(self, target_pred, target_true):
        self.output = mse_forward(target_pred, target_true)
        return self.output
    
    def backward(self, target_pred, target_true):
        return mse_grad_input(target_pred, target_true)

Let's check the progress one more time

In [ ]:
am.get_progress(username)

# Part two: Experiments

Seems like you've already implemented all the building blocks of the Neural Network. Now we will conduct several experiments.

**Note:** These experiments will not be evaluated.

## 9. Circles Classification Task

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image
from IPython.core.display import HTML 

In this first task we created a dataset of 2 circles made out of dots from 2 different formulas. It is your task to predict for each dot which circle it belongs to. 

The main purpose of this task is to understand the **importance of network design and parameter tuning (number of layers, number of hidden units, etc.)**. At first, we will generate and visualize the data in following cell.

In [ ]:
# Generate some data
N = 100
phi = np.linspace(0.0, np.pi * 2, 100)
X1 = 1.1 * np.array([np.sin(phi), np.cos(phi)])
X2 = 3.0 * np.array([np.sin(phi), np.cos(phi)])

Y = np.concatenate([np.ones(N), -1.0 * np.ones(N)]).reshape((-1, 1))

X = np.hstack([X1,X2]).T
plt.scatter(X[:,0], X[:,1], c=Y.ravel(), edgecolors= 'none')

As you have already written the code blocks, we will just call those functions and will train the network to classify the two circles. 

For this task we have provided the code for training and testing of the network in following blocks. Students are asked to design the network with different configurations  and observe the outputs.
* **Single Layer Neural Network**
* **Multiple Layer Neural Network**
* **Different number of Hidden units**
* **With and without activation function**

In [ ]:
##Training the network ##
model = SequentialNN()

###YOUR CODE FOR DESIGNING THE NETWORK ###
# model.add(...)
# model.add(...)

loss = Hinge()
weight_decay = 0.01
sgd = SGD(model, lr=0.1, weight_decay=weight_decay)
epochs = 100 # Number of times to iterate over all data objects

model.set_training_phase(True)

for i in range(100):
    # get the predictions
    y_pred = model.forward(X)
    
    # compute the loss value + L_2 term
    loss_value = loss.forward(y_pred, Y) + l2_regularizer(weight_decay, model.get_params())
    
    # log the current loss value
    print('Step: {}, \tLoss = {:.2f}'.format(i+1, loss_value))
    
    # get the gradient of the loss functions
    loss_grad = loss.backward(y_pred, Y)

    # backprop the gradients
    model.backward(X, loss_grad)
    
    # perform the updates
    sgd.update_params()

In [ ]:
##Testing the network ##
model.set_training_phase(False)
y_pred = model.forward(X) > 0
plt.scatter(X[:,0], X[:,1], c = y_pred.ravel())

# 10. Digits Classification Task

In this task you will implement a neural network for classification of hand written digits. You can use the blocks of code which you implemented in the first part of this assignment for completing this task.

We will use **digits** dataset for this task. This dataset consists of 1797 8x8 images. Further information about the dataset can be found [here](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits). 

In [ ]:
import sklearn.datasets

# We load the dataset
digits = sklearn.datasets.load_digits()

# Here we load up the images and labels and print some examples
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:10]):
    plt.subplot(2, 5, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap='gray_r', interpolation='nearest')
    plt.title('Training: {}'.format(label), y=1.1)

Next we will divide the images and labels data into two parts i.e. training data and test data.

In [ ]:
n_objects = digits.images.shape[0]
train_test_split = 0.7
train_size = int(n_objects * train_test_split)
indices = np.arange(n_objects)
np.random.shuffle(indices)

train_indices, test_indices = indices[:train_size], indices[train_size:]
train_images, train_targets = digits.images[train_indices], digits.target[train_indices]
test_images, test_targets = digits.images[test_indices], digits.target[test_indices]

In [ ]:
plt.imshow(train_images[0], cmap='gray_r')

The images in the dataset are $8 \times 8$ and each pixels in the image is an integer between ranging from 0 to 16. Before giving the images as input to the neural network we will reshape them to a 1-dimensional vector of 64 features as shown in the figure below.
![in](./src/image_pixel_input.png)

In [ ]:
train_images = train_images.reshape((-1, 64))
test_images = test_images.reshape((-1, 64))

train_targets = train_targets.reshape(-1, 1)
test_targets = test_targets.reshape(-1, 1)

The basic units of the neural network are perceptrons. A perceptron consists of a cell with atleast two inputs. The cell takes the inputs multiplied with weights and gives an output after computing the values. The basic diagram of a cell is shown below.
![neuron](./src/neuron.png)
For the image dataset which we will be using in this task the perceptron will have 64 inputs and 64 weights.
![N_weights](./src/weights.png)
As the digits dataset consists of 10 classes (0 to 9), in order to classifiy the images we will need 10 neurons for the prediction of the target class as shown in the image below. Each neuron will give an output and the output from the neuron with the highest value will be selected and that will be the predicted class.
![NN](./src/design.png)
Now, in order to perform classification for images you will use the functions which you implemented before.

In the following lines of code we will train a complete Neural Network by giving the images as input and the labels as targets to the network. At first we will design the network by setting the parameters of the network by calling the *SequentialNN()* function. After that, we will forward propagate the inputs through the network and calculate the loss, the error between the predicted output and the target output. This loss will then be back propagated through the network. Finally the parameters of the network will be updated in the right direction to reduce the error of the prediction.

In [ ]:
### YOUR CODE FOR TRAINING THE NETWORK###

#Specify the input size for the network
#specify the output size for the network
#specify the inputs for the network
#specify the outputs for the network
num_input = None
num_output = None
X = None
Y = None

model = SequentialNN()
# model.add(...)
# model.add(...)
# etc...

loss = Hinge()
weight_decay = 0.01
sgd = SGD(model, lr=0.01, weight_decay=weight_decay)
model.set_training_phase(True)

for i in range(100):
    # get the predictions
    y_pred = model.forward(X)
    
    # compute the loss value + L_2 term
    loss_value = loss.forward(y_pred, Y) + l2_regularizer(weight_decay, model.get_params())
    
    # log the current loss value
    print('Step: {}, \tLoss = {:.2f}'.format(i+1, loss_value))
    
    # get the gradient of the loss functions
    loss_grad = loss.backward(y_pred, Y)
    # backprop the gradients
    model.backward(X, loss_grad)
    
    # perform the updates
    sgd.update_params()

After training the network should be tested on test data; images in this task. During the test time unlabeled inputs are given to the network and by using the trained weights from the training cycle of the network the ouput classes for the unlabeled inputs are predicted. The figure below shows the difference between the training and the testing of the network.
![test](./src/Test.PNG)
In the following cell implement the code for testing the network. 

In [ ]:
#Testing the network 
###YOUR CODE FOR TESTING THE NETWORK  ###

For further practice and advanced experiments you can use MNIST dataset for the classification of hand written digits. It is a commonly used image dataset for testing machine learning algorithms. You can load this dataset by using the code from the file *data_utils* in week_1 folder. 

In [ ]:
am.get_progress(username)